How to use gulp?

In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('gulp');
    require.resolve('gulp-runtime');
    require.resolve('gulp-replace');
    require.resolve('gulp-tap');
} catch (e) {
    execSync('npm install "github:gulpjs/gulp#4.0" gulp-runtime gulp-replace gulp-tap');
}
var gulp = require('gulp');
var tap = require('gulp-tap');
var replace = require('gulp-replace');

// rename the project
gulp.task('build :src :dest', function (files, project) {
    return gulp
        .src(files, {cwd: project})
        .pipe(replace('appId: \'my-app-id\'', 'appId: \'jupytangular-module-server\''))
        .pipe(tap(function (file) {
            console.log(file.contents.toString());
        }))
});

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = PROFILE_PATH + '/Documents/universal';

gulp.task('default', () => gulp.task(['build :src :dest'])(['**/app.browser.module.ts'], project));

$$.async()
try {
    gulp.series(() => gulp.task(['default'])(), function (done) {
        done();
        $$.sendResult();
    })();
} catch (e) {
    $$.sendError(e);
}

How to convert a gulp task to a Promise?

How to use a Promise as a gulp task?



In [ ]:
var gulp = require('gulp');

// I wish Linus Torvolds had absolute power over everything javascript(+derivatives) and he could just do a massive purge of all the bullshit.
function tasksToPromise(tasks) {
    return new Promise((resolve, reject) => {
        gulp.series.apply(gulp, tasks.concat([function (done) {
            resolve(done());
        }]))();
    });
};
module.exports = tasksToPromise;
tasksToPromise;



In [ ]:
var fs = require('fs');
var path = require('path');
var replace = require('gulp-replace');
var gulp = require('gulp');

// add automock to project for callback services
var client, renderer, server, core;

function applyUniversal(project) {
    // gulp.task('theme',
    //    gulp.src('./some/dir/**/*.js', {cwd: 'path/to/project'})
    //        .src('./specific-file.ts')) // already within cwd in chain?

    // TODO: copy auth, resolve dependencies using file system script, generate code below

    // replace package.json
    gulp.task(
        'project relatives',
        gulp.series(projectRelatives(project)))
    // in *.js Can't resolve 'socket.io-client/package'
    // in *.ts Cannot find name 'JwtHelper' -> trace back to import statements?


    gulp.task('server module', function () {
        return gulp.src(['**/server/app/app.server.module.js'], {cwd: project})
        // add mock socket server to server/render-server or app.server.module
            .pipe(replace('import { AppComponent } from \'./app.component\'', `
import { AppComponent } from './app.component';
import { AuthService } from '../../client/app/auth/auth.service';
import { LogService } from '../../imports/log.service';
import { AuthManager } from '../../client/app/auth/auth.manager';
import { JwtHelper } from 'angular2-jwt/angular2-jwt';
import { MailgunValidatorService } from '../../client/app/auth/mailgun-validate.service';
`))
            // TODO: pipe server.ts to add log provider for log service
            .pipe(replace(
                /\)\],\s*AppModule\s*,providers\s*:\s*\[/ig,
                `)],
        AppModule
    ],
    providers: [
        {
            provide: AuthService,
            useValue: require('./sockify-server').sockifyRequire(AuthService, 'AuthService')
        },
        {
            provide: PlatformLocation,
            useValue: require('./sockify-server').sockifyRequire(PlatformLocation, 'PlatformLocation')
        },
        {
            provide: LogService,
            useValue: require('./sockify-server').sockifyRequire(LogService, 'LogService')
        },
        {
            provide: AuthGuard,
            useValue: require('./sockify-server').sockifyRequire(AuthGuard, 'AuthGuard')
        },
        AuthManager, JwtHelper, MailgunValidatorService,

    `))
            .pipe(gulp.dest(project))
    });

    // add socket-client provider for auth module to app.browser.module
    gulp.task('browser module', function () {
        return gulp.src(['**/client/app/app.browser.module.ts'], {cwd: project})
            .pipe(replace('import { AppComponent } from \'./app.component\'', `
import { AppComponent } from './app.component';
import { AuthService } from './auth/auth.service';
import { PlatformLocation } from '@angular/common';
import { LogService } from '../../imports/log.service';
import { AuthManager } from './auth/auth.manager';
import { JwtHelper } from 'angular2-jwt/angular2-jwt';
import { MailgunValidatorService } from './auth/mailgun-validate.service';
`))
            .pipe(replace(
                /\)\],\s*AppModule/ig,
                `)],
        AppModule,
    ],
    providers: [ AuthManager, JwtHelper, MailgunValidatorService,
        {
            provide: AuthService,
            useValue: require('./sockify-client').sockifyClient(AuthService, 'AuthService'),
        },
        {
            provide: PlatformLocation,
            useValue: require('./sockify-client').sockifyClient(PlatformLocation, 'PlatformLocation'),
        },
`))
            .pipe(gulp.dest(project))
    });

    // replace routes with lazy modules from other projects
    gulp.task('app router', function () {
        return gulp.src(['**/client/app/app.routes.ts'], {cwd: project})
            .pipe(replace(
                /children: \[/ig,
                `children: [
            {
                path: 'auth',
                loadChildren: './auth/auth.module#AuthModule'
            },
`))
            .pipe(gulp.dest(project))
    });

    // TODO: try new versions of packages to act like greenkeeper.io

    // gulp apply everything
    gulp.task('apply universal',
        gulp.series('server module', 'browser module''app router'));

    return tasksToPromise(['apply universal'], []);
};
applyUniversal;



Apply universal to angular project?

In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = PROFILE_PATH + '/Documents/universal';
if (!fs.existsSync(project)) {
    fs.mkdirSync(project);
}

$$.async()
applyUniversal(project)
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Fix project relatives as a service for updating files when they change?



In [ ]:
var gulp = require('gulp'),
    watch = require('gulp-watch');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = PROFILE_PATH + '/Documents/universal';

var searching = false;

gulp.task('watch for changes', function () {
    // Callback mode, useful if any plugin in the pipeline depends on the `end`/`flush` event 
    return watch('src/**/*.ts', {
        cwd: project,
        ignoreInitial: true,
        read: false,
        readDelay: 100
    }, function () {
        if (!searching) {
            searching = true;
            projectRelatives(project).then(() => {
                searching = false
            });
        }
    });
});

$$.async();
gulp.task(['watch for changes'])();



Search notebooks for a string using gulp?

search notebooks?



In [ ]:
var importer = require('../Core');
var path = require('path');
var gulp = require('gulp');
var tap = require('gulp-tap');
var JSONStream = require('JSONStream');
var Duplex = require('stream').Duplex;
var tasksToPromise = importer.import('gulp tasks to promise', {gulp});

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(__dirname, '..');

function bufferToStream(buffer) {
    let stream = new Duplex();
    stream.push(buffer);
    stream.push(null);
    return stream;
}

function searchNotebooks(search) {
    var cells;
    gulp.task('search notebooks', function () {
        cells = [];
        return gulp.src(
            ['**/*.ipynb', '!**/node_modules/**', '!**/\\.*'],
            {cwd: project})
            .pipe(tap(file => {
                var matches = bufferToStream(file.contents)
                    .pipe(JSONStream.parse([true, {emitPath: true}]));
                var accumulatedMarkdown = [], counter = 0;
                matches.on('data', match => {
                    var c = match.value;
                    if (match.path[0] === 'cells') {
                        counter++;
                        var source = c.source.join('');
                        if (c.cell_type === 'markdown') {
                            accumulatedMarkdown.push(source);
                        } else if (c.cell_type !== 'code') {
                            return;
                        }

                        if (source.match(new RegExp(search, 'ig'))) {
                            cells[cells.length] = {
                                filename: file.path,
                                code: source,
                                markdown: accumulatedMarkdown,
                                from: counter - 1,
                                to: counter,
                                language: c.language
                            };
                        }
                        accumulatedMarkdown = [];
                    }
                });
            }));
    });

    return tasksToPromise(['search notebooks'], [])
        .then(() => cells);
};
module.exports = searchNotebooks;



In [1]:
var importer = require('../Core');
var searchNotebooks = importer.import('search notebooks using gulp');
var searchRegexp = 'permissions';

$$.async();
searchNotebooks(searchRegexp)
    .then(r => $$.mime({'text/markdown': '```\n' + r.map(c => c.code).join('') + '\n```\n-------------\n'}))
    .catch(e => $$.sendError(e));


```
# Make ssh dir
RUN mkdir /root/.ssh/

# Copy over private key, and set permissions
ADD id_rsa /root/.ssh/id_rsa

# Create known_hosts
RUN touch /root/.ssh/known_hosts
var importer = require('../Core');
var searchNotebooks = importer.import('search notebooks using gulp');
var searchRegexp = 'permissions';

$$.async();
searchNotebooks(searchRegexp)
    .then(r => $$.mime({'text/markdown': '```\n' + r.map(c => c.code).join('') + '\n```\n-------------\n'}))
    .catch(e => $$.sendError(e));
var bodyParser = require('body-parser');
var express = require('express');
var importer = require('../Core')
var {
    getPermissions,
    getParameterNames
} = importer.import([
    'rpc permissions',
    'function parameters'
]);

var app = express();
var server = require('http').createServer(app);
app.use(bodyParser.json());    // to support JSON-encoded bodies
app.use(bodyParser.urlencoded({// to support URL-encoded bodies
    extended: true
}));

var router = express.Router();

// get permissions
var rpcFunctions = Object.keys(getPermissions())
    .filter(k => getPermissions()[k].indexOf('Public') > -1);

// rpc endpoints
// TODO: ? https://en.wikipedia.org/wiki/Portmap
console.log(rpcFunctions);

function addRoute(search) {
    const parameters = getParameterNames(search);
    console.log(parameters);
    router.all('/' + parameters[0], (req, res) => {
        res.header('access-control-allow-origin', req.headers['origin']);
        res.header('access-control-allow-headers', 'Authorization, X-Requested-With, origin, content-type, accept')
        res.header('Accept-Control-Allow-Methods', 'POST,GET,DELETE,PUT,OPTIONS');
        if(req.method === 'OPTIONS') {
            return res.send('');
        }
        const result = importer.import(search);
        console.log('Starting ' + parameters.join(' '))
        return Promise.resolve(result.apply(importer, parameters.slice(1).map(k => req.query[k] || req.body[k])))
            .then(r => res.send(JSON.stringify(r, null, 4)))
            // TODO: object assign error?
            .catch(e => res.send(e + ''));
    })
}

for(var i = 0; i < rpcFunctions.length; i++) {
    addRoute(rpcFunctions[i]);
}

app.use(router);

// open the port
if(typeof listener !== 'undefined') {
    listener.close();
}
console.log('Listening on 0.0.0.0:18135');
var listener = server.listen(18135).on('error', e => {
    if(e.code !== 'EADDRINUSE') {
        throw e;
    }
});

// shut down properly
process.on ('SIGTERM', () => {listener.close(); process.exit(0)});
process.on ('SIGINT', () => {listener.close(); process.exit(0)});

if(typeof $$ !== 'undefined') {
    $$.done();
}
rpc permissions?

// Always return the permission set so it can't be manipulated by another command.
function getPermissions() {
    return {
        'google contacts.ipynb[google contact settings]': ['Inner Circle'],
        'import notebook.ipynb[load interpreter]' : ['Public'],
        'rpc.ipynb[permissions]' : ['Public'],
        'rpc.ipynb[parameter names]' : ['Public'],
        'gulp.ipynb[search notebooks]' : ['Public'],
        'data collection.ipynb[search all]' : ['Public'],
        'data collection.ipynb[get joke]' : ['Public'],
    };
}
module.exports = getPermissions;
var importer = require('../Core');
var {
    getPermissions,
    getSettings,
    getDaysEvents,
    getOauthClient,
} = importer.import([
    'rpc permissions',
    'days events',
    'google contact settings',
    'import google calendar api'
]);

var options = {
    calendarId: 'Commands'
};

// TODO: move this logic out to a higher level coordinator?
function filterCommand(command, date, id, user) {
    const props = {};
    return getOauthClient(options)
        .then(() => getSettings(user))
        .then(settings => {
            // filter permissions
            Object.assign(props, settings);
            try {
                console.log(settings.circles)
                // compare id with permissions
                Object.assign(props, {
                    result: importer.interpret(command)
                });
                Object.assign(props, {
                    allowed: settings.circles
                        .filter(c => {
                            const matching = Object.keys(getPermissions())
                                .filter(k => getPermissions()[k].indexOf(c) > -1 || getPermissions()[k].indexOf('Public') > -1);
                            return matching.filter(k => importer.interpret(k).id === props.result.id).length > 0;
                        }).length > 0
                });
                // if looking for settings use the user name provided to this function for lookup
                // TODO: accept parameters from message context
                if(props.result.id === importer.interpret('google contacts.ipynb[google contact settings]').id) {
                    props.result.runInNewContext = () => settings;
                }
            } catch (e) {
                if((e + '').indexOf('Nothing found') > -1) {
                    Object.assign(props, {result: null});
                } else {
                    throw e;
                }
            }
        })
        .then(() => getDaysEvents(new Date(date), options))
        .then(events => {
            // filter already sent
            const already = events
              .filter(e => e.event.summary.indexOf('Result:') > -1 && e.event.summary.indexOf(id) > -1).length > 0;
            Object.assign(props, {already: already})
        })
        .catch(e => console.log(e))
        .then(() => props)
}
module.exports = filterCommand;
var cors = require('cors')({origin: true});
var importer = require('../Core');
var functions = require('firebase-functions'),
    admin = require('firebase-admin');

try {
    admin.initializeApp(functions.config().firebase);
} catch (e) {
    console.log(e);
}

var {
    getPermissions,
    getParameterNames
} = importer.import([
    'rpc permissions',
    'function parameters'
]);
var rpcFunctions = Object.keys(getPermissions())
    .filter(k => getPermissions()[k].indexOf('Public') > -1);

console.log(rpcFunctions);

if(typeof module.exports === 'undefined') {
    module.exports = {};
}

function addRoute(search) {
    const parameters = getParameterNames(search);
    console.log(parameters);
    module.exports[parameters[0]] = functions.https.onRequest((req, res) => {
        cors(req, res, () => {
            // TODO: get parameters from query
            const result = importer.import(search);
            console.log('Starting ' + parameters.join(' '))
            return Promise.resolve(result.apply(importer, parameters.slice(1).map(k => req.query[k] || req.body[k])))
                .then(r => res.send(JSON.stringify(r, null, 4)))
                // TODO: object assign error?
                .catch(e => res.send(e + ''));
        });
    });
}

for(var i = 0; i < rpcFunctions.length; i++) {
    addRoute(rpcFunctions[i]);
}
var importer = require('../Core');
var path = require('path');
var fs = require('fs');
var execCmd = importer.import('spawn child process');

var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads');
var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.defaultProfile');
var DOCKERFILE = path.resolve(path.join(__dirname, 'Dockerfile'));

function getSeleniumServer(name = 'act-selenium') {
    try {
        fs.mkdirSync(DOWNLOAD_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    try {
        fs.mkdirSync(PROFILE_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    try {
        fs.unlinkSync(path.join(PROFILE_DIR, 'SingletonLock'));
    } catch (err) {
        if (err.code != 'ENOENT') {
            throw err;
        }
    }
    seleniumDocker(DOCKERFILE);
    return execCmd('docker ps -a')
        .then(r => {
            if (r[0].indexOf(name) > -1) {
                return execCmd('docker stop ' + name)
                    .then(r => new Promise(resolve =>
                        setTimeout(() => resolve(r), 1000)))
                    .then(() => execCmd('docker rm ' + name));
            }
        })
        .then(() => new Promise(resolve =>
            setTimeout(() => resolve(), 1000)))
        .then(() => {
            var build = 'docker build -t ' + name + ' "'
                + path.dirname(DOCKERFILE) + '"\n'
                + 'docker run --shm-size=3g -d '
                + '--name ' + name + ' '
                + '-p 8888:8888 '
                + '-p 6080:6080 '
                + '-p 5900:5900 '
                + '-p 4444:4444 '
                + '-p 4200:4200 '
                + '-p 3000:3000 '
                // TODO: add profile dir back in when permissions works on windows
                + '-v "' + DOWNLOAD_DIR + '":/data/downloads '
                + name + '\n';
            return execCmd(build)
        })
        .then(r => new Promise(resolve => setTimeout(() => resolve(r), 6000)))
        .catch(e => console.log(e))
};
module.exports = getSeleniumServer;
getSeleniumServer;


```
-------------


Find relative paths and includes?

In [ ]:
var fs = require('fs');
var path = require('path');
var gulp = require('gulp');
var tap = require('gulp-tap');
var importer = require('../Core');
var taskToPromise = importer.import('gulp.ipynb[Promise gulp task]', {gulp})

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/portal') : project;

var relatives = [];
var packages = [];
var missing = {};
var missingPackages = [];

gulp.task('get relative imports', function () {
    return gulp.src(['**/*.+(ts|js|ipynb)'], {
        ignore: ['**/node_modules/**',
                '**/packages/**',
                '**/vendor/**',
                '/Users/**/Downloads/**',
                '/Users/**/Library/**',
                '/Users/**/Applications/**',
                '/Users/**/Music/**',
                '**/wp-content/**',
                '**/wp-includes/**',
                '**/Pods/**',
                '**/svn/**',
                '**/.git/**',
                '**/.vscode/**',
                '**/.npm/**',
                '**/\\.*',
                '**/.*',
                '**/Cache/**',
                '**/Creative Cloud Files/**'],
        cwd: project
    })
        .pipe(tap(function (file) {
        try {
            var content = file.contents.toString();
            var re = /(from|require)[\s\\(]*["'](.*?)['"][\\\s\)]*/ig;
            var m;
            while ((m = re.exec(content))) {
                if (m[2].match(/^\./)) {
                    relatives[relatives.length] = file.path + '/' + m[2];
                    var res = path.resolve(path.join(path.dirname(file.path), m[2]));
                    if (!fs.existsSync(res) && !fs.existsSync(res + '.ts')
                        && !fs.existsSync(res + '.js') && !fs.existsSync(res + '.ipynb')) {
                        var key = path.basename(m[2]);
                        if (typeof missing[key] !== 'undefined') {
                            missing[key][missing[key].length] = file.path + '/' + m[2];
                        } else {
                            missing[key] = [file.path + '/' + m[2]];
                        }
                    }
                } else {
                    packages[packages.length] = file.path + '/' + m[2];
                    try {
                        require.resolve(m[2]);
                    } catch (e) {
                        if (missingPackages.indexOf(m[2]) === -1) {
                            missingPackages[missingPackages.length] = file.path + '/' + m[2];
                        }
                    } finally {
                    }
                }
            }
        } catch (e) {
            console.log(e);
        }
    }))
});

function getRelativeImports(p) {
    relatives = [];
    packages = [];
    missing = {};
    missingPackages = [];
    project = p;
    return taskToPromise(['get relative imports'])
        .then(r => ({relatives, packages, missing, missingPackages}))
        .catch(e => console.log(e))
};
module.exports = getRelativeImports;


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/universal') : project;
var importer = require('../Core');
var projectRelatives = importer.import('fix project paths');


$$.async();
projectRelatives(project)
    //.then(r => console.log(r))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));

